__NAME:__ __FULLNAME__  
__SECTION:__ __NUMBER__  
__CS 5970: Machine Learning Practices__

# Homework 7: Model Comparisons

## Assignment Overview
Generally, it's helpful to first read through the entire notebook before writing
any code to obtain a sense of the overall program structure before you start coding.  

Follow the TODOs and read through and understand any provided code.  


### Task
For this assignment, you'll be comparing different models after performing holistic
cross validation to find the best parameter sets for various sizes of the training
data.


### Data set
The BMI data will be utilized. Recall:  
* _MI_ files contain data with the number of activations for 48 neurons, at mutliple time points, for a single fold. There are 20 folds (20 files), where each fold consists of over 1000 times points (the rows). At each time point, we record the number of activations for each neuron for 20 bins. Therefore, each time point has 48 * 20 = 960 columns.  
* _theta_ files record the angular position of the shoulder (in column 0) and the elbow (in column 1) for each time point. 
* _dtheta_ files record the angular velocity of the shoulder (in column 0) and the elbow (in column 1) for each time point.  
* _torque_ files record the torque of the shoulder (in column 0) and the elbow (in column 1) for each time point. 
* _time_ files record the actual time stamp of each time point.  


### Objectives
* Understanding regularization using __holistic cross validation__
* Training set size sensitivity analysis
* Model selection

### Notes
* Do not save work within the ml_practices folder

### General References
* [Guide to Jupyter](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook)
* [Python Built-in Functions](https://docs.python.org/3/library/functions.html)
* [Python Data Structures](https://docs.python.org/3/tutorial/datastructures.html)
* [Numpy Reference](https://docs.scipy.org/doc/numpy/reference/index.html)
* [Numpy Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)
* [Summary of matplotlib](https://matplotlib.org/3.1.1/api/pyplot_summary.html)
* [DataCamp: Matplotlib](https://www.datacamp.com/community/tutorials/matplotlib-tutorial-python?utm_source=adwords_ppc&utm_campaignid=1565261270&utm_adgroupid=67750485268&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=332661264365&utm_targetid=aud-299261629574:dsa-473406587955&utm_loc_interest_ms=&utm_loc_physical_ms=9026223&gclid=CjwKCAjw_uDsBRAMEiwAaFiHa8xhgCsO9wVcuZPGjAyVGTitb_-fxYtkBLkQ4E_GjSCZFVCqYCGkphoCjucQAvD_BwE)
* [Pandas DataFrames](https://urldefense.proofpoint.com/v2/url?u=https-3A__pandas.pydata.org_pandas-2Ddocs_stable_reference_api_pandas.DataFrame.html&d=DwMD-g&c=qKdtBuuu6dQK9MsRUVJ2DPXW6oayO8fu4TfEHS8sGNk&r=9ngmsG8rSmDSS-O0b_V0gP-nN_33Vr52qbY3KXuDY5k&m=mcOOc8D0knaNNmmnTEo_F_WmT4j6_nUSL_yoPmGlLWQ&s=h7hQjqucR7tZyfZXxnoy3iitIr32YlrqiFyPATkW3lw&e=)
* [Sci-kit Learn Linear Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Sci-kit Learn Ensemble Models](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble)
* [Sci-kit Learn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)
* [Sci-kit Learn Model Selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
* [SciPy Paired t-test for Dependent Samples](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ttest_rel.html)
* [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples)
* [Understanding Paired t-tests](https://machinelearningmastery.com/how-to-code-the-students-t-test-from-scratch-in-python/)

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import os, re, fnmatch
import pathlib, itertools, time
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.externals import joblib


FIGW = 10
FIGH = 6
FONTSIZE = 12

HOME_DIR = pathlib.Path.home()

plt.rcParams['figure.figsize'] = (FIGW, FIGH)
plt.rcParams['font.size'] = FONTSIZE

plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE

%matplotlib inline

In [ ]:
"""
Display current working directory of this notebook. If you are using 
relative paths for your data, then it needs to be relative to the CWD.
"""
pathlib.Path.cwd()

# LOAD DATA

In [ ]:
def read_bmi_file_set(directory, filebase):
    '''
    Read a set of CSV files and append them together
    :param directory: The directory in which to scan for the CSV files
    :param filebase: File specification potentially including wildcards
    :returns: A list of Numpy arrays (one for each fold)
    '''
    
    # The set of files in the directory
    files = fnmatch.filter(os.listdir(directory), filebase)
    files.sort()

    # Create list of Pandas objects; 
    # Each from a file in the directory matching the filebase
    lst = [pd.read_csv(directory + "/" + file, delim_whitespace=True).values 
           for file in files]
    
    # Concatenate the Pandas objects together. ignore_index is 
    # critical here so that the duplicate row indices are addressed
    return lst

In [ ]:
""" PROVIDED
Load the BMI data from all the folds, using read_bmi_file_set()
"""
# TODO: might need to change directory
dir_name = str(HOME_DIR / 'ml_practices/imports/datasets/bmi/DAT6_08')

MI_folds = read_bmi_file_set(dir_name, 'MI_fold*')
theta_folds = read_bmi_file_set(dir_name, 'theta_fold*')
dtheta_folds = read_bmi_file_set(dir_name, 'dtheta_fold*')
torque_folds = read_bmi_file_set(dir_name, 'torque_fold*')
time_folds = read_bmi_file_set(dir_name, 'time_fold*')

alldata_folds = zip(MI_folds, theta_folds, dtheta_folds, 
                    torque_folds, time_folds)

nfolds = len(MI_folds)
nfolds

In [ ]:
""" PROVIDED
Print out the shape of all the data for each fold
"""
for i, (MI, theta, dtheta, torque, time) in enumerate(alldata_folds):
    print("FOLD %2d " % i, MI.shape, theta.shape, 
          dtheta.shape, torque.shape, time.shape)

# PARAMETER SET LIST

In [ ]:
def generate_paramsets(param_lists):
    '''
    Construct the Cartesian product of the parameters
    PARAMS:
        params_lists: dict of lists of values to try for each parameter.
                      keys of the dict are the names of the parameters
                      values are lists of values to try for the 
                      corresponding parameter
    RETURNS: a list of dicts that make up the Cartesian product of the 
    parameters
    '''
    keys, values = zip(*param_lists.items())
    # Determines cartesian product of parameter values
    combos = itertools.product(*values)
    # Constructs list of dictionaries
    combos_dicts = [dict(zip(keys, vals)) for vals in combos]
    return list(combos_dicts)

# PERFORMANCE EVALUTION

In [ ]:
def mse_rmse(trues, preds):
    '''
    Compute MSE and rMSE for each column separately.
    '''
    mse = np.sum(np.square(trues - preds), axis=0) / trues.shape[0]
    rmse_rads = np.sqrt(mse)
    rmse_degs = rmse_rads * 180 / np.pi
    return mse, rmse_rads, rmse_degs

def score_eval(model, X, y, preds):
    '''
    Compute the model predictions and corresponding scores, for an
    already trained model.
    PARAMS:
        model: model to predict with
        X: input feature data
        y: true output for X
        preds: predicted output for X
    RETURNS: results as a dictionary of numpy arrays
        mse: mean squared error for each column
        rmse_rads: rMSE in radians
        rmse_deg: rMSE in degrees
        evar: explained variance, best is 1.0
        score: score computed by the models score() method
    '''
    score = model.score(X, y)

    mse, rmse_rads, rmse_degs = mse_rmse(y, preds)
    evar = explained_variance_score(y, preds)
    
    # Dictionary of numpy arrays. The numpy arrays must
    # be row vectors, where each element is the result 
    # for a different output, when using multiple regression.
    # The keys of the dictionary are the name of the performance 
    # metric, and the values are the numpy row vectors
    results = {'mse': np.reshape(mse, (1, -1)), 
               'rmse_rads': np.reshape(rmse_rads, (1, -1)), 
               'rmse_degs': np.reshape(rmse_degs, (1, -1)), 
               'evar': np.reshape(evar, (1, -1)), 
               'score': np.reshape(score, (1, -1)), 
              }
    return results

# CROSS VALIDATION

In [ ]:
""" TODO: 
FILL IN WITH YOUR SOLUTION FROM HW6 for perform_cross_validation(). All 
that needs to be done here is simply copy/paste your code from HW6 into
perform_cross_validation()
"""
class KFoldHolisticCrossValidation():
    def __init__(self, model, paramsets, eval_func, opt_metric, 
                 maximize_opt_metric=False, trainsizes=[1], rotation_skip=1):
        ''' 
        Object for managing and performing cross validation for a given 
        model for a list of parameter sets and train set sizes. Note, 
        train set size is in terms of number of folds (not samples)
        
        General Procedure:
        + iter over hyper-parameter sets
          1. set hyper-parameters of the model
          2. iter over train set sizes
             a. iter over splits/rotations
                  i. train the model
                 ii. evaluate the model on train, val, and test sets
                iii. record the results
             b. record the results by size
          3. record the results by hyper-parameter set

        PARAMS:
            model: base ML model
            
            paramsets: list of dicts of parameter sets to give to the model
            
            eval_func: handle to function used to evaluate/score the model
                       The eval_func definition must have the following  
                       arguments: model, X, ytrue, ypreds; and return a dict 
                       of numpy arrays with shape 1-by-n, where n is the
                       number of outputs if using multiple regression.
                       template function header: 
                           def eval_func(model, X, y, preds)
                       template output: 
                           {'metrics1':1_by_n_array, ...}
                       
            opt_metric: the optimized metric. one of the metric key names 
                        returned from eval_func to use to pick the best 
                        parameter sets
                        
            maximize_opt_metric: True if opt_metric is maximized; 
                                 False if minimized
            
            trainsizes: list of training set sizes (in number of folds) to try
            
            rotation_skip: build model and evaluate every ith rotation (1=all 
                           possible rotations; 2=every other rotation, etc.)
        ''' 
        self.model = model
        self.paramsets = paramsets
        self.trainsizes = trainsizes
        self.eval_func = eval_func
        self.opt_metric = opt_metric + '_mean'
        self.maximize_opt_metric = maximize_opt_metric
        self.rotation_skip = rotation_skip
        
        # Results attributes
        # Full recording of all results for all paramsets, sizes, rotations,
        # and metrics. This is a list of dictionaries for each paramset
        self.results = None
        # Validation summary report of all means and standard deviations for 
        # all metrics, for all paramsets, and sizes. This is a 3D s-by-r-by-p 
        # numpy array. Where s is the number of sizes, r the number of summary 
        # metrics +2, and p is the number of paramsets
        self.report_by_size = None
        # List of the indices of the best paramset for each size
        self.best_param_inds = None

    def perform_cross_validation(self, all_Xfolds, all_yfolds, 
                                 trainsize, verbose=0):
        ''' TODO: FILL IN WITH YOUR SOLUTION FROM HW6
        Perform cross validation for a singular train set size and single 
        hyper-parameter set, by evaluating the model's performance over 
        multiple data set rotations all of the same size.

        NOTE: This function assumes the hyper-parameters have already been 
              set in the model
            
        PARAMS:
            all_Xfolds: list containing all of the input data folds
            all_yfolds: list containing all of the output data folds
            trainsize: number of folds to use for training
            verbose: flag to display simple debugging information
            
        RETURNS: train, val, and test set results for all rotations of the 
                 data sets and the summary (i.e. the averages over all the 
                 rotations) of the results. 
                 results is a dictionary of dictionaries of r-by-n numpy 
                 arrays. Where r is the number of rotations, and n is the 
                 number of outputs from the model.
                 summary is a dictionary of dictionaries of 1-by-n numpy 
                 arrays. 

                 General form:
                     results.keys() = ['train', 'val', 'test']

                     results['train'].keys() = ['metric1', 'metric2', ...]
                     
                     results['train']['metric1'] = numpy_array
                     
                     results = 
                     {
                        'train':
                                 {
                                     'mse'      : r_by_n_numpy_array,
                                     'rmse_rads': r_by_n_numpy_array, 
                                     'rmse_degs': r_by_n_numpy_array,
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }
                     
                     summary = 
                     {
                        'train':
                                 {
                                     'mse_mean'      : 1_by_n_numpy_array,
                                     'mse_std'       : 1_by_n_numpy_array,
                                     'rmse_rads_mean': 1_by_n_numpy_array, 
                                     'rmse_rads_std' : 1_by_n_numpy_array,
                                     ...
                                 },
                        'val'  : {...},
                        'test' : {...}
                     }

                    For example, you can access the MSE results for the 
                    validation set like so:
                        results['train'][metric] 
                    For example, you can access the summary (i.e. the average 
                    results over all the rotations) for the test set for the
                    rMSE in degrees like so:
                        summary['test']['rmse_degs_mean']                
        '''
        
        # Verify a valid train set size was provided
        nfolds = len(all_Xfolds)
        if trainsize < 1 or trainsize > nfolds - 2: 
            err_msg = "ERROR: KFoldHolisticCrossValidation.perform_cross_validation() - "
            err_msg += "trainsize (%d) must be between 1 and nfolds (%d) - 2" % (trainsize, nfolds)
            raise ValueError(err_msg)
            
        # Verify rotation skip
        if self.rotation_skip < 1: 
            err_msg = "ERROR: KFoldHolisticCrossValidation.__init__() - "
            err_msg += "rotation_skip (%d) can't be less than 1" % self.rotation_skip
            raise ValueError(err_msg)
        
        # Set up results recording for each rotation
        results = {'train': None, 'val': None, 'test': None}
        summary = {'train': {}, 'val': {}, 'test': {}}
        
        model = self.model
        evaluate = self.eval_func
        
        # Rotate through different train, val, and test sets
        for rotation in range(0, nfolds, self.rotation_skip):
            # Determine fold indices for train, val, and test set. 
            # The val and tests are each only 1 fold
            
            
        
            # Construct train set by concatenating the individual  
            # training folds
            
            

            # Construct validation set
            
            
            
            # Construct test set
            
            
            
            # DEBUGGING
            if verbose:
                print("TRAIN", X.shape, y.shape, trainfolds)
                print("VAL", Xval.shape, yval.shape, valfold)
                print("TEST", Xtest.shape, ytest.shape, testfold)
            
            # Train model using the training set
            
            
            
            # Predict with the model for train, val, and test sets
            
            
            
            # Evaluate the model for each set
            
            

            # Record the train, val, and test set results. These are dicts 
            # of result metrics, returned by the evaluate function
            # For the first rotation, store the results from evaluating
            # with the train, val, and tests by setting the values of   
            # the appropriate items within the results dict
            if results['train'] is None: 
                
                
                
            else:
                # Append the results for each rotation
                for metric in res_train.keys():
                    results['train'][metric] = np.append(results['train'][metric], 
                                                         res_train[metric], axis=0)
                    results['val'][metric] = np.append(results['val'][metric], 
                                                       res_val[metric], axis=0)
                    results['test'][metric] = np.append(results['test'][metric], 
                                                        res_test[metric], axis=0)

        # Compute/record mean and standard deviation for the size for each metric
        for metric in results['train'].keys():
            for stat_set in ['train', 'val', 'test']:
                summary[stat_set][metric+'_mean'] = np.mean(results[stat_set][metric], 
                                                            axis=0).reshape(1, -1)
                summary[stat_set][metric+'_std'] = np.std(results[stat_set][metric], 
                                                          axis=0).reshape(1, -1)

        return results, summary

    def grid_cross_validation(self, all_Xfolds, all_yfolds, verbose=0):
        '''
        (MAIN PROCEDURE) Perform cross validation for multiple sets of 
        parameters and train set sizes. Calls self.perform_cross_validation(). 
        This is the procedure that executes cross validation for all parameter 
        sets and all sizes.
        
        General Procedure:
        + iter over hyper-parameter sets
          1. set hyper-parameters of the model
          2. iter over train set sizes
             a. iter over splits/rotations
                  i. train the model
                 ii. evaluate the model on train, val, and test sets
                iii. record the results
             b. record the results by size
          3. record the results by hyper-parameter set
        
        PARAMS:
            all_Xfolds: all the input data folds (list of folds, as it was 
                        loaded from the files)
            all_yfolds: all the output data folds (list of folds)
            verbose: flag to print out simple debugging information
            
        RETURNS: best parameter set for each train set size as a list of 
                 parameter indices. Additionally, returns self.report_by_size,
                 the 3D array of validation means (overall rotations) for all 
                 paramsets, for each metric, for all sizes. The structure of 
                 the returned object is a dictionary of the following form: 
                 { 
                   'report_by_size' : self.report_by_size, 
                   'best_param_inds': self.best_param_inds
                 }
        ''' 
        sizes = self.trainsizes
        paramsets = self.paramsets
        nparamsets = len(paramsets)
        print("nparamsets", nparamsets)
        
        # Set up all results
        all_results = []
        
        # Iterate over parameter sets
        for params in paramsets:
            # Set up paramset results 
            param_res = []
            param_smry = None
            
            # Set model parameters
            print("Current paramset\n", params)
            self.model.set_params(**params)

            # Iterate over the different train set sizes
            for size in sizes:
                # Cross-validation for current model and train size
                res, smry = self.perform_cross_validation(all_Xfolds, 
                                                          all_yfolds, 
                                                          size, verbose)

                # Save the results
                param_res.append(res) 
                # Save the mean and standard deviation statistics (summary)
                if param_smry is None: param_smry = smry
                else:
                    # For each metric measured, append the summary results
                    for metric in smry['train'].keys():
                        for stat_set in ['train', 'val', 'test']:
                            stat = smry[stat_set][metric]
                            param_smry[stat_set][metric] = np.append(param_smry[stat_set][metric],
                                                                     stat, axis=0)
            
            # Append the results and summary for the parameter set
            all_results.append({'params':params, 'results':param_res, 
                                'summary':param_smry})
        
        # Generate reports and determine best params for each size 
        self.results = all_results
        self.report_by_size = self.get_reports()
        self.best_param_inds = self.get_best_params(self.opt_metric, 
                                                    self.maximize_opt_metric)
        return {'report_by_size':self.report_by_size, 
                'best_param_inds':self.best_param_inds}

    def get_reports(self):
        ''' 
        Get the mean validation summary of all the parameters for each size
        for all metrics. This is used to determine the best parameter set  
        for each size
        
        RETURNS: the report_by_size as a 3D s-by-r-by-p array. Where s is 
                 the number of train sizes tried, r is the number of summary  
                 metrics evaluated+2, and p is the number of parameter sets.
        '''
        results = self.results
        sizes = np.reshape(self.trainsizes, (1, -1))
        
        nsizes = sizes.shape[1]
        nparams = len(results)
        
        # Set up the reports objects
        metrics = list(results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics 
        report_by_size = np.empty((nsizes, len(colnames), nparams), dtype=object)

        # Determine mean val for each paramset for each size for all metrics
        for p, paramset_result in enumerate(results):
            params = paramset_result['params']
            res_val = paramset_result['summary']['val']

            # Compute mean val result for each train size for each metric
            means_by_size = [np.mean(res_val[metric], axis=1) 
                             for metric in metrics]
            # Include the train set sizes into the report
            means_by_size = np.append(sizes, means_by_size, axis=0)
            # Include the parameter sets into the report
            param_strgs = np.reshape([str(params)]*nsizes, (1, -1))
            means_by_size = np.append(param_strgs, means_by_size, axis=0).T
            # Append the parameter set means into the report 
            report_by_size[:,:,p] = means_by_size
        return report_by_size

    def get_best_params(self, opt_metric, maximize_opt_metric):
        ''' 
        Determines the best parameter set for each train size,  
        based on a specific metric.
        
        PARAMS:
            opt_metric: optimized metric. one of the metrics returned 
                        from eval_func, with '_mean' appended for the
                        summary stat. This is the mean metric used to  
                        determine the best parameter set for each size
                        
            maximize_opt_metric: True if the max of opt_metric should be
                                 used to determine the best parameters.
                                 False if the min should be used.
        RETURNS: list of best parameter set indicies for each size 
        '''
        results = self.results
        report_by_size = self.report_by_size 
                
        metrics = list(results[0]['summary']['val'].keys())
        
        # Determine best params for each size, for the optimized metric
        best_param_inds = None
        metric_idx = metrics.index(opt_metric)
        
        # Report info for all paramsets for the optimized metric
        report_opt_metric = report_by_size[:, metric_idx+2, :]
        
        if maximize_opt_metric:
            # Add two for the additional cols for params and size
            best_param_inds = np.argmax(report_opt_metric, axis=1)
        else: 
            best_param_inds = np.argmin(report_opt_metric, axis=1)
        # Return list of best params indices for each size
        return best_param_inds
    
    def get_best_params_strings(self):
        ''' 
        Generates a list of strings of the best params for each size
        RETURNS: list of strings of the best params for each size
        '''
        best_param_inds = self.best_param_inds
        results = self.results
        return [str(results[p]['params']) for p in best_param_inds]

    def get_report_best_params_for_size(self, size):
        ''' 
        Get the mean validation summary for the best parameter set 
        for a specific size for all metrics.
        PARAMS:
            size: index of desired train set size for the best  
                  paramset to come from. Size here is the index in 
                  the trainsizes list, NOT the actual number of folds.
        RETURNS: the best parameter report for the size as an s-by-m  
                 dataframe. Where each row is for a different size, and 
                 each column is for a different summary metric.
        '''
        best_param_inds = self.best_param_inds
        report_by_size = self.report_by_size 

        # Obtain the index of the best parameter set for the size
        bp_index = best_param_inds[size]

        # Obtain the list of metrics
        metrics = list(self.results[0]['summary']['val'].keys())
        colnames = ['params', 'size'] + metrics
        
        # Create DataFame with all summary stats for the parameter set
        report_best_params_for_size = pd.DataFrame(report_by_size[:,:,bp_index],
                                                   columns=colnames)
        return report_best_params_for_size

    def plot_cv(self, foldsindices, results, summary, metrics, size):
        ''' 
        Plotting function for after perform_cross_validation(), 
        displaying the train and val set performances for each rotation 
        of the training set. 
        
        PARAMS:
            foldsindices: indices of the train sets tried
            results: results from perform_cross_validation()
            summary: mean and standard deviations of the results
            metrics: list of result metrics to plot. Available metrics 
                     are the keys in the dict returned by eval_func
            size: train set size
            
        RETURNS: the figure and axes handles
        '''
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        fig.subplots_adjust(hspace=.35)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            # Compute the mean for multiple outputs
            res_train = np.mean(results['train'][metric], axis=1)
            res_val = np.mean(results['val'][metric], axis=1)
            #res_test = np.mean(results['test'][metric], axis=1)
            # Plot
            ax.plot(foldsindices, res_train, label='train')
            ax.plot(foldsindices, res_val, label='val')
            #ax.plot(foldsindices, res_test, label='test')
            ax.set(ylabel=metric)
        axs[0].legend(loc='upper right')
        axs[0].set(xlabel='Fold Index')
        axs[0].set(title='Performance for Train Set Size ' + str(size))
        return fig, axs

    def plot_param_train_val(self, metrics, paramidx=0, view_test=False):
        ''' 
        Plotting function for after grid_cross_validation(), 
        displaying the mean (summary) train and val set performances 
        for each train set size.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std'
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
            paramidx: parameter set index
            view_test: flag to view the test set results
            
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results

        summary = results[paramidx]['summary']
        params = results[paramidx]['params']
        
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(12,6))
        fig.subplots_adjust(hspace=.35)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            # Compute the mean for multiple outputs
            res_train = np.mean(summary['train'][metric], axis=1)
            res_val = np.mean(summary['val'][metric], axis=1)
            # Plot
            ax.plot(sizes, res_train, label='train')
            ax.plot(sizes, res_val, label='val')
            if view_test:
                res_test = np.mean(summary['test'][metric], axis=1)
                ax.plot(sizes, res_test, label='test')
            ax.set(ylabel=metric)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title=str(params))
        axs[0].legend(loc='upper right')
        return fig, axs
    
    def plot_allparams_val(self, metrics):
        ''' 
        Plotting function for after grid_cross_validation(), displaying  
        mean (summary) validation set performances for each train size 
        for all parameter sets for the specified metrics.
        
        PARAMS:
            metrics: list of summary metrics to plot. '_mean' or '_std' 
                     must be append to the end of the base metric name. 
                     These base metric names are the keys in the dict 
                     returned by eval_func
                     
        RETURNS: the figure and axes handles
        '''
        sizes = self.trainsizes
        results = self.results
        
        nmetrics = len(metrics)

        # Initialize figure plots
        fig, axs = plt.subplots(nmetrics, 1, figsize=(10,6))
        fig.subplots_adjust(hspace=.35)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()

        # Construct each subplot
        for metric, ax in zip(metrics, axs):
            for p, param_results in enumerate(results):
                summary = param_results['summary']
                params = param_results['params']
                # Compute the mean for multiple outputs
                res_val = np.mean(summary['val'][metric], axis=1)                
                ax.plot(sizes, res_val, label=str(params))
            ax.set(ylabel=metric)
        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].set(title='Validation Performance')
        axs[0].legend(bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 8})
        return fig, axs

    def plot_best_params_by_size(self):
        ''' 
        Plotting function for after grid_cross_validation(), displaying 
        mean (summary) train and validation set performances for the best 
        parameter set for each train size for the optimized metric.
                     
        RETURNS: the figure and axes handles
        '''
        results = self.results
        metric = self.opt_metric
        best_param_inds = self.best_param_inds
        sizes = np.array(self.trainsizes)

        # Unique set of best params for the legend
        unique_param_sets = np.unique(best_param_inds)
        lgnd_params = [self.paramsets[p] for p in unique_param_sets]

        # Initialize figure
        fig, axs = plt.subplots(2, 1, figsize=(10,6))
        fig.subplots_adjust(hspace=.35)
        # When 1 metric is provided, allow the axs to be iterable
        axs = np.array(axs).ravel()
        set_names = ['train', 'val']

        # Construct each subplot
        for i, (ax, set_name) in enumerate(zip(axs, set_names)):
            for p in unique_param_sets:
                # Obtain indices of sizes this paramset was best for
                param_size_inds = np.where(best_param_inds == p)[0]
                param_sizes = sizes[param_size_inds]
                # Compute the mean over multiple outputs for each size
                param_summary = results[p]['summary'][set_name]
                metric_scores = np.mean(param_summary[metric][param_size_inds, :], axis=1)
                # Plot the param results for each size it was the best for
                ax.scatter(param_sizes, metric_scores, s=120, marker=(p+2, 1))
                #ax.grid(True)

            set_name += ' Set Performance'
            ax.set(ylabel=metric, title=set_name)

        axs[-1].set(xlabel='Train Set Size (# of folds)')
        axs[0].legend(lgnd_params, bbox_to_anchor=(1.02, 1), loc='upper left',
                      ncol=1, borderaxespad=0., prop={'size': 7})
        return fig, axs

# PERFORM CROSS VALIDATION
Initialize holistic cross validation objects to explore Linear, Ridge, Lasso, and ElasticNet models.

The experiments for the ElasticNet have been provided in a file (hw7_full_crossval.pkl) due to the length of time it takes to run; however, you are welcome to re-run these experiements, for all/various train set sizes, and rotations, using score_eval as the eval_func, and rmse_degs as the metric to optimize. The file can be found in the hw7 folder in the ml_practices directory, along with this notebook.

The inputs for the models are the MI data and the outputs are the torque (you'll provide the shoulder and elbow simulataneouly, as done in the previous HW).

In [ ]:
""" PROVIDED
Holistic Cross Validation Options:
* ridge_alphas: list of alphas to try for the RIDGE model
* lasso_alphas: list of alphas to try for the LASSO model
* en_alphas: list of alphas to try for the ELASTICNET model
* l1_ratios: list of l1_ratios to try for the ELASTICNET model

* trainsizes: list of number of folds to utilize in the train set
* opt_metric: the optimized metric, returned by the eval_func, used 
  to select the best parameter sets
* maximize_opt_metric: True if the opt_metric is maximized; False 
  otherwise
* skip: the number of folds to skip when rotating through train sets 
  of the same size
"""
ridge_alphas = [1, 10, 50, 100, 500, 1000, 10000]
lasso_alphas = [.001, .005, .01, .025, .05, .075, .1]
en_alphas = lasso_alphas + [0.5, 1]
l1_ratios = [0.001, .025, .05, .1, .5, 1]

trainsizes = range(1, nfolds-1)
opt_metric = 'rmse_degs'
maximize_opt_metric = False
skip = 1

# True to always run cross validation, false to re-load existing run
# or run cross validation for the first time
force = False 
# Tag for the filename to save the experiments to
prefix = "_full" 

## [LINEAR REGRESSION](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Ordinary least squares Linear Regression.

In [ ]:
""" TODO
LinearRegression

Execute cross validation procedure for all sizes for the 
LinearRegression model using grid_cross_validation().
The parameter list for the LinearRegression model is a
list with just an empty dictionary [{}]
"""
lnr_fullcvfname = "hw7" + prefix + "_linear_crossval.pkl"

model = LinearRegression()
lnr_crossval = KFoldHolisticCrossValidation(model, [{}], score_eval, 
                                            opt_metric, maximize_opt_metric,
                                            trainsizes, skip)

lnr_crossval_report = None
if force or (not os.path.exists(lnr_fullcvfname)):
    # TODO: Execute cross validation procedure for all parameters and sizes
    lnr_crossval_report = # TODO
    # TODO: Save the cross validation object, use joblib.dump()
    
    
else:
    # Re-load saved crossval object instead of re-running 
    lnr_crossval = joblib.load(lnr_fullcvfname)
    lnr_crossval_report = {'report_by_size': lnr_crossval.report_by_size,
                         'best_param_inds': lnr_crossval.best_param_inds}

lnr_crossval.model, lnr_crossval.rotation_skip, lnr_crossval.trainsizes

## [RIDGE](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

$\min_w ||y - w^TX||^2_2 + \alpha ||w||^2_2$

$\alpha$: amount of $L_2$ regularization to apply. Larger $\alpha$ greater penalize the model for larger weights

$w$: the weights from the model

$X$: feature or input data

$y$: true outputs


In [ ]:
""" TODO
RIDGE

Initialize a KFoldHolisticCrossValidation object that uses RIDGE
as the model, and the provided r_allparamsets

Execute cross validation procedure for all sizes for the Ridge
model using grid_cross_validation()
"""
r_fullcvfname = "hw7" + prefix + "_ridge_crossval.pkl"

r_param_lists = {'alpha':ridge_alphas, 'max_iter':[1e4]}
r_allparamsets = generate_paramsets(r_param_lists)
print(pd.DataFrame(r_allparamsets))

model = Ridge()
# TODO: Initialize a KFoldHolisticCrossValidation object using Ridge
r_crossval = # TODO

r_crossval_report = None
if force or (not os.path.exists(r_fullcvfname)):
    # TODO: Execute cross validation for all parameters and sizes
    r_crossval_report = # TODO
    # TODO: Save the cross validation object
    

else:
    # Re-load saved crossval object instead of re-running 
    r_crossval = joblib.load(r_fullcvfname)
    r_crossval_report = {'report_by_size' : r_crossval.report_by_size,
                         'best_param_inds': r_crossval.best_param_inds}

r_crossval.model, r_crossval.rotation_skip, r_crossval.trainsizes

## [LASSO](https://scikit-learn.org/stable/modules/linear_model.html#lasso)

$\min_w \frac{1}{2 N} ||y - w^TX||^2_2 + \alpha ||w||_1$

$N$: the number of samples


In [ ]:
""" TODO
LASSO

Initialize a KFoldHolisticCrossValidation object that uses LASSO
as the model, and the provided l_allparamsets

Execute cross validation procedure for all sizes for the Lasso
model using grid_cross_validation()
"""
l_fullcvfname = "hw7" + prefix + "_lasso_crossval.pkl"

l_param_lists = {'alpha':lasso_alphas, 'max_iter':[1e4]}
l_allparamsets = generate_paramsets(l_param_lists)
print(pd.DataFrame(l_allparamsets))

model = Lasso()
# TODO: Initialize a KFoldHolisticCrossValidation object using Lasso
l_crossval = # TODO

l_crossval_report = None
if force or (not os.path.exists(l_fullcvfname)):
    # TODO: Execute cross validation for all parameters and sizes
    l_crossval_report = # TODO
    # TODO: Save the cross validation object
    
    
else:
    # Re-load saved crossval object instead of re-running 
    l_crossval = joblib.load(l_fullcvfname)
    l_crossval_report = {'report_by_size' : l_crossval.report_by_size,
                         'best_param_inds': l_crossval.best_param_inds}

l_crossval.model, l_crossval.rotation_skip, l_crossval.trainsizes

## [ELASTICNET](https://scikit-learn.org/stable/modules/linear_model.html#elastic-net)

$\min_w \frac{1}{2 N} ||y - w^TX||^2_2 + \alpha L_1 ||w||_1 + \frac{1}{2} \alpha (1 - L_1) ||w||^2_2$

$L_1$: the $L_1$ ratio

In [ ]:
""" TODO
ELASTICNET

Initialize a KFoldHolisticCrossValidation object that uses ELASTICNET
as the model, and the provided allparamsets

Execute cross validation procedure for all sizes for the ELASTICNET
model using grid_cross_validation()

Re-load the existing experiment
"""
fullcvfname = "hw7" + prefix + "_crossval.pkl"

param_lists = {'alpha':en_alphas, 'l1_ratio':l1_ratios, 'max_iter':[1e4]}
allparamsets = generate_paramsets(param_lists)
nparamsets = len(allparamsets)
print(pd.DataFrame(allparamsets))

model = ElasticNet()
crossval = KFoldHolisticCrossValidation(model, allparamsets, score_eval, 
                                        opt_metric, maximize_opt_metric,
                                        trainsizes, skip)

crossval_report = None
if force or (not os.path.exists(fullcvfname)):
    # Execute cross validation for all parameters and sizes
    crossval_report = crossval.grid_cross_validation(MI_folds, 
                                                     torque_folds, 
                                                     verbose=0)
    # Save the cross validation object
    joblib.dump(crossval, fullcvfname)
else:
    # TODO: Re-load saved crossval object. Use joblib.load()
    crossval = # TODO
    crossval_report = {'report_by_size' : crossval.report_by_size, 
                       'best_param_inds': crossval.best_param_inds}

crossval.model, crossval.rotation_skip, crossval.trainsizes

# RESULTS

### Understand the result output structure

In [ ]:
""" PROVIDED
List KFoldHolisticCrossValidation Attributes
"""
dir(crossval)

In [ ]:
""" PROVIDED
Results attribute is a list of dictionaries. Each element, or dictionary
corresponds to the results for a single parameter set
"""
len(crossval.results), crossval.results[0].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'] is a list of dictionaries with the results
  for each size for the parameter set at index 0
* crossval.results[1]['summary'] is a dictionary of summary results for the 
  train, val, and test sets for the parameter set at index 1
"""
len(crossval.results[0]['results']), crossval.results[1]['summary'].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2] is a dictionary with the results
  for the train size at index 2 for the parameter set at index 0
* crossval.results[1]['summary']['val'] is a dictionary of summary (over the 
  sizes) results for the val set for the parameter set at index 1, for all 
  metrics
"""
crossval.results[0]['results'][2].keys(), crossval.results[1]['summary']['val'].keys()

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2]['train'] is a dictionary of all results for 
  the train set for the parameter set at index 0, the size at index 2, for all
  metrics
* crossval.results[1]['summary']['val']['mse_mean'] is a numpy array of averages 
  for the val set for the parameter set at index 1, for the mse. The averages are 
  computed over the sizes
"""
crossval.results[0]['results'][2]['train'].keys(), crossval.results[1]['summary']['val']['mse_mean'].shape

In [ ]:
""" PROVIDED
* crossval.results[0]['results'][2]['train']['mse'] is a dictionary of all 
  results for the train set for the parameter set at index 0, the size at 
  index 2, for the mse, for all rotations (there are 20 rotations when skip=1)
"""
crossval.results[0]['results'][2]['train']['mse'].shape

### Best Parameters for Each Size

In [ ]:
""" PROVIDED
Results options:
* size_idx: index of the size from the list of train sizes to examine results
* metrics: list of summary (average) metrics to examine results
"""
# index 7 corresponds to train size 8
size_idx = 7 
metrics = ['rmse_degs_mean', 'evar_mean']

In [ ]:
""" PROVIDED
Display the lists of the best parameter sets for each size for all
the models, expect the Linear model (as it has only one parameter set)
"""
print("Best Parameter Sets For Each Train Set Size")

print("RIDGE")
r_best_param_info = pd.DataFrame((r_crossval.trainsizes, 
                                  r_crossval.best_param_inds, 
                                  r_crossval.get_best_params_strings()),
                                  index=['train_size','param_index','paramset'])
print(r_best_param_info.T)


print("LASSO")
l_best_param_info = pd.DataFrame((l_crossval.trainsizes, 
                                  l_crossval.best_param_inds, 
                                  l_crossval.get_best_params_strings()),
                                  index=['train_size','param_index','paramset'])
print(l_best_param_info.T)

print("ELASTICNET")
best_param_info = pd.DataFrame((crossval.trainsizes, 
                                crossval.best_param_inds, 
                                crossval.get_best_params_strings()),
                                index=['train_size', 'param_index', 'paramset'])
print(best_param_info.T)

### Plot Best Parameters for Each Size

In [ ]:
""" PROVIDED
LINEAR REGRESSION
Plot the mean (summary) train and validation set performances for 
each train size for the optimized metric. Use plot_best_params_by_size()

Note: for LinearRegression, there is only one parameter set.
"""
lnr_crossval.plot_best_params_by_size() 

In [ ]:
""" TODO
RIDGE
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""



In [ ]:
""" TODO
LASSO
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""



In [ ]:
""" TODO
ELASTICNET
Plot the mean (summary) train and validation set performances for 
the best parameter set for each train size for the optimized
metrics. Use plot_best_params_by_size()
"""



### Plot Validation for All Parameter Sets for Each Size

In [ ]:
""" TODO
LINEAR REGRESSION
Plot the validation results for all parameter sets over all train 
sizes, for the specified metrics, rmse_degs_mean and evar_mean
(this variable is declared above). Use plot_allparams_val()
"""



In [ ]:
""" TODO
RIDGE
Plot the validation results for all parameter sets over all train 
sizes, for the specified metrics, rmse_degs_mean and evar_mean
(this variable is declared above). Use plot_allparams_val()
"""



In [ ]:
""" TODO
LASSO
Plot the validation results for all parameter sets over all train 
sizes, for the specified metrics, rmse_degs_mean and evar_mean
(this variable is declared above). Use plot_allparams_val()
"""



In [ ]:
""" TODO
ELASTICNET
Plot the validation results for all parameter sets over all train 
sizes, for the specified metrics, rmse_degs_mean and evar_mean
(this variable is declared above). Use plot_allparams_val()
"""



### Plot the TRAIN and VAL Set Performances

In [ ]:
""" TODO
LINEAR REGRESSION
For the best parameter set for the train set size at 
size_idx=7 (this variable has already been declared above),
plot the TRAIN and VAL set performances using 
plot_param_train_val() for just the optimized metric.

Note: there is only one parameter set for the Linear model, 
thus paramidx=0
"""
print("Train Set Size", trainsizes[size_idx])



In [ ]:
""" TODO
RIDGE
For the best parameter set for the train set size at 
size_idx=7 (this variable has already been declared above),
plot the TRAIN and VAL set performances using 
plot_param_train_val() for just the optimized metric

Use r_crossval.best_param_inds to get the desired parameter 
set index
"""
print("Train Set Size", trainsizes[size_idx])



In [ ]:
""" TODO
LASSO
For the best parameter set for the train set size at 
size_idx=7 (this variable has already been declared above),
plot the TRAIN and VAL set performances using 
plot_param_train_val() for just the optimized metric
"""
print("Train Set Size", trainsizes[size_idx])



In [ ]:
""" 
ELASTICNET
For the best parameter set for the train set size at 
size_idx=7 (this variable has already been declared above),
plot the TRAIN and VAL set performances using 
plot_param_train_val() for just the optimized metric
"""
print("Train Set Size", trainsizes[size_idx])
bp_idx = crossval.best_param_inds[size_idx]
crossval.plot_param_train_val([crossval.opt_metric], paramidx=bp_idx)

### Plot Performance over the Parameter Space

In [ ]:
def plot_param_val_for_size(crossval, metric, alphas, sizeidx=0):
    ''' PROVIDED
    Plotting function for after grid_cross_validation(), 
    displaying the mean (summary) train and val set performances 
    for each alpha, given the size, for RIDGE and LASSO only

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        alphas: list of alpha values
        sizeidx: train size index

    RETURNS: the figure and axes handles
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds

    nalphas = len(alphas)

    nsizes = len(sizes)
    nmetrics = len(metrics)

    # Initialize the matrices for the curve
    Y_train = np.empty((nalphas,))
    Y_val = np.empty((nalphas,))

    # Obtain the mean performance for the curve
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric][sizeidx, :])
        Y_train[alpha_idx] = res_train

        res_val = np.mean(summary['val'][metric][sizeidx, :])
        Y_val[alpha_idx] = res_val
    
    # Initialize figure plots
    fig = plt.figure(figsize=(12,2))
    for i, (Y, set_name) in enumerate(zip((Y_train, Y_val), 
                                          ('Training', 'Validation'))):
        # Plot
        ax = fig.add_subplot(1, 2, i+1)
        ax.plot(alphas, Y)
        title = "%s Performance, Train Size %d Folds" % (set_name, sizes[sizeidx])
        ax.set(title=title)
        ax.set(xlabel=r"$\alpha$", ylabel=metric)
    return fig


In [ ]:
def plot_surface(xlist, ylist, Z_train, Z_val, ylabel, zlabel, 
                 elev=30, angle=45, title_suffix=""):
    ''' PROVIDED
    Helper plotting function. x-axis is always alpha
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        xlist: list of x values
        ylist: list of y values
        Z_train: matrix of performance results from the training set
        Z_val: matrix of performance results from the validation set
        ylabel: y-axis label 
        zlabel: z-axis label
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view
        title_suffix: string to append to each subplot title

    RETURNS: the figure and axes handles
    '''
    # Initialize figure
    fig = plt.figure(figsize=(15,5))
    X, Y = np.meshgrid(xlist, ylist) 
    for i, (Z, set_name) in enumerate(zip((Z_train, Z_val), 
                                          ('Training', 'Validation'))):
        # Plot the surface
        ax = fig.add_subplot(1, 2, i+1, projection='3d')
        surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, 
                               linewidth=0, antialiased=False)
        title = "%s Performance %s" % (set_name, title_suffix)
        ax.view_init(elev=elev, azim=angle)
        ax.set(title=title)
        ax.set(xlabel=r"$\alpha$", ylabel=ylabel, zlabel=zlabel)
    return fig

In [ ]:
def plot_param_val_surface_RL(crossval, metric, alphas, elev=30, angle=245):
    ''' PROVIDED
    Plotting function for after grid_cross_validation(), 
    displaying the mean (summary) train and val set performances 
    for each alpha, for all sizes, for RIDGE and LASSO only
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        alphas: list of alpha values
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view

    RETURNS: the figure and axes handles
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds

    nalphas = len(alphas)

    nsizes = len(sizes)
    nmetrics = len(metrics)

    # Initialize the matrices for the surface
    Z_train = np.empty((nsizes, nalphas))
    Z_val = np.empty((nsizes, nalphas))

    # Obtain the mean performance for the surface
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric], axis=1)
        Z_train[:, alpha_idx] = res_train

        # Compute the mean for multiple outputs
        res_val = np.mean(summary['val'][metric], axis=1)
        Z_val[:, alpha_idx] = res_val
    
    fig = plot_surface(alphas, sizes, Z_train, Z_val, 'size (# of folds)', 
                       metric, elev, angle)
    return fig

In [ ]:
def plot_param_val_surface_EN(crossval, metric, param_lists, 
                              sizeidx=0, elev=35, angle=280):
    ''' PROVIDED
    Plotting function for after grid_cross_validation(), 
    displaying the mean (summary) train and val set performances 
    for each alpha and l1_ratio, given the size, for the ELASTICNET
    
    REQUIRES: from mpl_toolkits.mplot3d import Axes3D

    PARAMS:
        crossval: cross validation object
        metric: summary metric to plot. '_mean' or '_std' must be 
                append to the end of the base metric name. These 
                base metric names are the keys in the dict returned
                by eval_func
        param_lists: dictionary of the list of alphas and l1_ratios
        sizeidx: train size index
        elev: elevation of the 3D plot for the view
        angle: angle in degrees of the 3D plot for the view

    RETURNS: the figure and axes handles
    '''
    sizes = crossval.trainsizes
    results = crossval.results
    best_param_inds = crossval.best_param_inds

    alphas = list(param_lists['alpha'])
    l1_ratios = list(param_lists['l1_ratio'])

    nalphas = len(alphas)
    nl1_ratios = len(l1_ratios)

    nsizes = len(sizes)
    nmetrics = len(metrics)

    # Initialize the matrices for the surface
    Z_train = np.empty((nl1_ratios, nalphas))
    Z_val = np.empty((nl1_ratios, nalphas))

    # Obtain the mean performance for the surface 
    for param_res in results:
        params = param_res['params']
        summary = param_res['summary']

        alpha_idx = alphas.index(params['alpha'])
        l1_idx = l1_ratios.index(params['l1_ratio'])

        # Compute the mean for multiple outputs
        res_train = np.mean(summary['train'][metric][sizeidx, :])
        Z_train[l1_idx, alpha_idx] = res_train

        res_val = np.mean(summary['val'][metric][sizeidx, :])
        Z_val[l1_idx, alpha_idx] = res_val
    
    fig = plot_surface(alphas, l1_ratios, Z_train, Z_val, 'l1_ratio', 
                       metric, elev, angle,', Size %d Folds' % sizes[sizeidx])
    return fig

In [ ]:
""" PROVIDED
List the parameter sets explored for RIDGE
"""
r_crossval.paramsets

In [ ]:
""" TODO
Plot the performance versus alpha for the RIDGE model
using plot_param_val_for_size() for size indices 0, 3, and 7, 
for the optimized metric (use r_crossval.opt_metric)
"""




In [ ]:
""" TODO
RIDGE
Use plot_param_val_surface_RL() to plot the surface of the training
and validation set performance versus alpha and size in the X and Y axes,
using the optimized metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 30
# Angle the plot is viewed
angle = 255

# TODO: Plot




In [ ]:
""" PROVIDED
List the parameter sets explored for LASSO
"""
l_crossval.paramsets

In [ ]:
""" TODO
Plot the performance versus alpha for the LASSO model
using plot_param_val_for_size() for size indices 0, 3, and 7, 
for the optimized metric
"""



In [ ]:
""" TODO
LASSO
Use plot_param_val_surface_RL() to plot the surface of the training
and validation set performance versus alpha and size in the X and Y axes,
using the optimized metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 30
# Angle the plot is viewed
angle = 255

# TODO: Plot



In [ ]:
""" PROVIDED
List the parameter sets explored for ELASTICNET
"""
crossval.paramsets

In [ ]:
""" TODO
ELASTICNET
Use plot_param_val_surface_EN() to plot the surface of the training
and validation set performance versus alpha and l1_ratio in the X 
and Y axes for the size indices of 0, 3, and 7, for crossval.opt_metric
"""
# Feel free to adjust these to understand the shape of the surface
# Elevation of the plot
elev = 25
# Angle the plot is viewed
angle = 280

# TODO: Plot



### Paired t-tests
We can use paired t-tests to assess statistical significant differences between the mean test set performances of the models

In [ ]:
""" PROVIDED
Obtain all the results for all the models
"""
# LinearRegression
lnr_all_results = lnr_crossval.results

# RIDGE
r_all_results = r_crossval.results

# LASSO
l_all_results = l_crossval.results

# ELASTICNET
all_results = crossval.results

In [ ]:
""" TODO
Complete the plotting code

Plot distributions of the Validation and Test scores from the
best parameter set for each base model for the corresponding 
size indices, [0, 3, 7]. The metric of interest is rmse_degs.
These are the distribution of results from each rotation of 
the training set
"""
metric = 'rmse_degs'
set_names = ['val', 'test']
nbins = 11

# Size indices
size_indices = [0, 3, size_idx]

for si in size_indices:
    # Obtain the index of the best parameter set for the size
    # RIDGE
    r_bp_idx = r_crossval.best_param_inds[si]
    # LASSO
    l_bp_idx = l_crossval.best_param_inds[si]
    # ELASTICNET
    bp_idx = crossval.best_param_inds[si]

    # Construct the figure
    fig, axs = plt.subplots(2, 2, figsize=(15,7))
    for i, set_name in enumerate(set_names):
        title = '%s, Size %d' % (set_name, trainsizes[si])

        # LINEAR
        # Note: there's only 1 parameter set for the Linear model
        lnr_res = lnr_all_results[0]['results'][si][set_name]
        lnr_scores = np.mean(lnr_res[metric], axis=1)

        # RIDGE
        # Obtain results for the best parameter set for the size
        ridge_res = r_all_results[r_bp_idx]['results'][si][set_name]
        # Compute the mean of the outputs for each data set rotation
        ridge_scores = np.mean(ridge_res[metric], axis=1)

        # LASSO
        lasso_res = l_all_results[l_bp_idx]['results'][si][set_name]
        lasso_scores = np.mean(lasso_res[metric], axis=1)
        
        # ELASTICNET
        res = # TODO
        elastic_scores = # TODO
        
        # Determine the edges for the bins in the histograms
        all_scores = np.concatenate((elastic_scores, ridge_scores, 
                                     lasso_scores, lnr_scores))
        mn = np.min(all_scores)
        mx = np.max(all_scores)
        bins = np.linspace(mn, mx, nbins)

        # Histograms
        # TODO: include the hist of the elastic net scores 
        
        
        axs[0, i].hist(ridge_scores, bins=bins, alpha=.4)
        axs[0, i].hist(lasso_scores, bins=bins, alpha=.4)
        axs[0, i].hist(lnr_scores, bins=bins, alpha=.4)
        axs[0, i].legend(['ElasticNet', 'Ridge', 'Lasso', 'Linear'])
        axs[0, i].set(title=title, xlabel=metric)
        
        # Boxplots
        axs[1, i].boxplot([elastic_scores, ridge_scores, lasso_scores, lnr_scores])
        axs[1, i].set_xticklabels(['ElasticNet', 'Ridge', 'Lasso', 'Linear'])
        axs[1, i].set(ylabel=metric)


In [ ]:
""" TODO
Dependent Sample Paired t-test
Two-sided t-test for the null hypothesis that mean of the distribution
of differences between the two test performance distributions is zero 
"""
print("Train Set Size", trainsizes[size_idx])

# LINEAR
# Note: there's only 1 parameter set for the LinearRegression model
lnr_res = lnr_crossval.results[0]['results'][size_idx]['test']
lnr_test_res = np.mean(lnr_res[metric], axis=1)

# RIDGE
# Obtain index of best parameters for train size 8
r_bp_idx = r_crossval.best_param_inds[size_idx]
# Obtain all results for the best parameter set for train size 8
ridge_res = r_all_results[r_bp_idx]['results'][size_idx]['test']
# Compute the mean of the outputs for each data set rotation
ridge_test_res = np.mean(ridge_res[metric], axis=1)

# LASSO
l_bp_idx = l_crossval.best_param_inds[size_idx]
lasso_res = l_all_results[l_bp_idx]['results'][size_idx]['test']
lasso_test_res = np.mean(lasso_res[metric], axis=1)

# TODO: ELASTICNET
bp_idx = # TODO
net_res = # TODO
elastic_test_res = # TODO

In [ ]:
""" TODO
ELASTICNET vs RIDGE
Execute the paired t-test to determine whether to reject the null hypothesis 
(i.e. H0) with 95% confidence. H0 is that the mean of the distribution of the 
differences between test scores for the best ELASTICNET model and the best RIDGE 
is zero, when using a training size of 8 (i.e. the size at index 7 of the 
trainsizes list). Display the t-statistic, the p-value, and the mean of the 
differences (i.e. mean(elastic_test_res - ridge_test_res))

Use stats.ttest_rel(). See the API reference above.
Do the same for all the pairing of models
"""



In [ ]:
""" TODO
ELASTICNET vs LASSO
Execute the paired t-test
"""



In [ ]:
""" TODO
ELASTICNET vs LinearRegression
Execute the paired t-test
"""



In [ ]:
""" TODO
RIDGE vs LASSO
Execute the paired t-test
"""



In [ ]:
""" TODO
RIDGE vs LinearRegression
Execute the paired t-test
"""



In [ ]:
""" TODO
LASSO vs LinearRegression
Execute the paired t-test
"""



# DISCUSSION
For each question write 1 to 2 paragraphs of discussion:

1. Interpret the meaning of the t-test results using 95% confidence. Discuss the statistical meaning as well as the practical interpretation of the results in the context of the data set.


2. For the Elastic Net Model, discuss the differences in the surfaces between the train sizes of 1, 4, and 8 folds, for the training and validation sets.


3. For each of the train set sizes of 1, 4, and 8 folds, which model (Linear, Lasso, Ridge, or ElasticNet) and corresponding parameter set would you select and why? Specify which model and parameter set for each size. For each size, use plot_param_train_val() to view the train, val, and test sets of the chosen model(s). Remember, selections should be made based on the validation performance.


In [ ]:
""" TODO
Discussion question 3 plots
"""